In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
import sys
sys.path.append(os.getcwd() + '/..')

In [13]:
from absl import logging
logging._warn_preinit_stderr = 0
logging.warning('...')
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
from load_data_30_turn import load_all_data, load_test_data, load_saved_data
from tf_codes.models import get_model
from tf_codes.ENN_loss import build_evidential_cross_entropy, build_evidential_mse
import time

In [14]:
def get_model(algo, INPUT_LEN, dim, CLASSES):
    if algo == 'evidential_oos':
        return get_evidential_oos(INPUT_LEN, dim, CLASSES)
    elif algo == 'evidential_turn_30':
        return get_evidential_turn_30(INPUT_LEN, dim, CLASSES)
    else:
        print("model not implemented")
        return None


def get_evidential_oos(INPUT_LEN, dim, CLASSES):
    hidden_size = 128
    model = tf.keras.Sequential([
        # Seq2Seq
        Dense(hidden_size, input_shape=(INPUT_LEN*dim,), activation=relu),
        Dropout(0.2),
        Dense(CLASSES)
    ])
    return model


def get_evidential_turn_30(INPUT_LEN, dim, CLASSES):
    hidden_size = 128
    dropout = 0.05
    model = tf.keras.Sequential([
        # Seq2Seq
        Dense(hidden_size, input_shape=(INPUT_LEN*dim,), activation=relu),
        Dropout(dropout),
        Dense(hidden_size,  activation=relu),
        Dropout(dropout),
        Dense(hidden_size//2, activation=relu),
        Dropout(dropout),
        Dense(CLASSES, activation=relu) #
    ])
    return model


In [16]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [29]:
features = ['cog']
dim = len(features)
timesteps = 60 # number of sequences per batch
CLASSES = 2

In [32]:
# load_data
X_data = pd.read_csv('./tf_codes/X_data.csv',header=None)   # 60 timestep
Y_data = pd.read_csv('./tf_codes/Y_data.csv',header=None)

In [31]:
X_data

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,200.100006,198.899994,199.199997,199.699997,199.699997,199.199997,200.000000,200.100006,199.199997,199.600006,...,202.399994,202.600006,203.399994,206.000000,206.000000,206.000000,206.000000,205.600006,205.399994,205.199997
1,199.600006,199.600006,199.600006,199.199997,199.199997,199.199997,199.600006,200.000000,200.600006,200.100006,...,205.199997,205.800003,207.399994,209.199997,210.600006,211.199997,211.199997,211.399994,213.399994,214.800003
2,199.600006,200.500000,200.000000,199.699997,199.600006,200.000000,200.000000,200.399994,200.399994,200.399994,...,221.399994,222.600006,223.000000,223.300003,223.000000,222.199997,221.800003,222.199997,222.199997,222.199997
3,200.399994,202.199997,202.399994,202.600006,203.399994,206.000000,206.000000,206.000000,206.000000,205.600006,...,222.600006,222.800003,222.399994,223.000000,222.199997,222.199997,223.600006,223.800003,224.000000,224.500000
4,213.399994,214.800003,216.600006,217.000000,218.399994,217.199997,217.800003,220.399994,221.199997,222.399994,...,233.199997,234.199997,235.199997,235.600006,236.199997,237.600006,237.600006,239.000000,239.399994,240.399994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8963,29.400000,29.400000,29.400000,29.299999,29.299999,29.299999,29.299999,29.299999,29.299999,29.299999,...,30.900000,30.900000,30.900000,29.900000,29.900000,29.900000,29.900000,29.900000,29.900000,29.900000
8964,29.299999,29.299999,29.299999,29.900000,29.900000,29.900000,29.900000,29.900000,30.900000,29.900000,...,29.900000,30.900000,30.900000,30.900000,30.900000,31.500000,31.500000,31.500000,31.500000,31.400000
8965,29.900000,30.900000,29.900000,29.900000,28.900000,28.900000,29.900000,29.900000,29.900000,29.299999,...,31.500000,31.500000,31.400000,32.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
8966,29.299999,29.299999,28.900000,29.299999,29.299999,29.299999,29.299999,29.299999,29.299999,29.299999,...,31.000000,31.000000,31.000000,31.000000,31.000000,30.400000,30.400000,31.000000,30.400000,30.400000


In [27]:
Y_data

,0,1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
...,...,...
8963,1.0,0.0
8964,1.0,0.0
8965,1.0,0.0
8966,1.0,0.0
